# Solar Energy Optimization System - Demo
This notebook shows how to use the Solar Energy Optimization System.

In [ ]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
from src.forecast import EnergyDemandForecaster, SolarProductionForecaster
from src.optimizer import EnergyOptimizer
from src.utils import load_sample_data
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load Sample Data
Let's load the sample data for demonstration.

In [ ]:
# Load sample data
data = load_sample_data()
demand_df = data['demand']
solar_df = data['solar']
weather_df = data['weather']

# Display first few rows of each dataset
print("Demand Data:")
display(demand_df.head())

print("\nSolar Production Data:")
display(solar_df.head())

print("\nWeather Data:")
display(weather_df.head())

## 2. Train Forecasting Models
Let's train the demand and production forecasting models.

In [ ]:
# Initialize forecasters
demand_forecaster = EnergyDemandForecaster(model_type='prophet')
solar_forecaster = SolarProductionForecaster(model_type='prophet')

# Train models
print("Training demand forecaster...")
demand_forecaster.fit(demand_df[['timestamp', 'demand_kwh']])

print("Training solar production forecaster...")
solar_forecaster.fit(
    solar_df[['timestamp', 'solar_production_kwh']],
    weather_df[['timestamp', 'temperature', 'cloud_cover']]
)

print("Training complete!")

## 3. Generate Forecasts
Now, let's generate forecasts for the next 24 hours.

In [ ]:
# Generate timestamps for the next 24 hours
forecast_hours = pd.date_range(
    pd.Timestamp.now().normalize(),
    periods=24,
    freq='H'
)

# Generate demand forecast
demand_forecast = demand_forecaster.predict(forecast_hours)

# Generate solar production forecast
solar_forecast = solar_forecaster.predict(
    forecast_hours,
    weather_df[['timestamp', 'temperature', 'cloud_cover']]
)

# Plot the forecasts
plt.figure(figsize=(12, 6))
plt.plot(forecast_hours, demand_forecast, label='Demand Forecast')
plt.plot(forecast_hours, solar_forecast, label='Solar Production Forecast')
plt.title('24-Hour Energy Forecast')
plt.xlabel('Time')
plt.ylabel('Energy (kWh)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Optimize Energy Usage
Let's optimize the battery usage based on our forecasts.

In [ ]:
# Initialize the optimizer
optimizer = EnergyOptimizer(
    battery_capacity_kwh=10.0,
    max_charge_rate_kw=5.0,
    battery_efficiency=0.95,
    initial_soc=0.5
)

# Create electricity prices (higher during peak hours)
electricity_prices = pd.Series(
    data=0.15 + 0.1 * np.sin(np.linspace(-np.pi/2, 3*np.pi/2, 24)),
    index=forecast_hours
)

# Optimize the schedule
schedule = optimizer.optimize_schedule(
    demand_forecast=demand_forecast,
    production_forecast=solar_forecast,
    electricity_prices=electricity_prices
)

# Calculate savings
savings = optimizer.calculate_savings(schedule, electricity_prices)

# Display savings
print("Optimization Results:")
print(f"- Cost without optimization: ${savings['cost_without_optimization']:.2f}")
print(f"- Cost with optimization: ${savings['cost_with_optimization']:.2f}")
print(f"- Savings: ${savings['savings']:.2f} ({savings['savings_percent']:.1f}%)")
print(f"- Self-consumption ratio: {savings['self_consumption_ratio']*100:.1f}%")
print(f"- Autonomy ratio: {savings['autonomy_ratio']*100:.1f}%")

## 5. Visualize the Schedule
Let's visualize the optimized schedule.

In [ ]:
# Create a DataFrame for visualization
schedule_df = pd.DataFrame({
    'Battery Charge': schedule['battery_charge'],
    'Battery Discharge': schedule['battery_discharge'],
    'Grid Import': schedule['grid_import'],
    'Grid Export': -schedule['grid_export'],  # Negative for visualization
    'Battery SOC': schedule['battery_soc'] * 100  # Convert to percentage
}, index=forecast_hours)

# Plot the schedule
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [2, 1]})

# Plot energy flows
schedule_df[['Battery Charge', 'Battery Discharge', 'Grid Import', 'Grid Export']].plot.bar(
    stacked=True,
    ax=ax1,
    color=['green', 'red', 'blue', 'orange']
)
ax1.set_title('Optimized Energy Schedule')
ax1.set_ylabel('Energy (kWh)')
ax1.legend(title='Energy Flow')
ax1.grid(True, which='both', linestyle='--', alpha=0.7)

# Plot battery state of charge
schedule_df['Battery SOC'].plot(
    ax=ax2,
    color='purple',
    marker='o'
)
ax2.set_title('Battery State of Charge')
ax2.set_ylabel('SOC (%)')
ax2.set_ylim(0, 100)
ax2.grid(True, which='both', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 6. Run the Interactive Dashboard
For a more interactive experience, you can run the web-based dashboard.

In [ ]:
# Uncomment and run the following line to start the dashboard
# !python -m src.dashboard